In [11]:
%reload_ext autoreload
%autoreload 2

import glob
import os, gc
import numpy as numpy
import pandas as pd
import scipy as sp
import datatable as dt
from collections import defaultdict
from tqdm.notebook import tqdm
from sklearn.utils import shuffle
from sklearn.metrics import r2_score
from numba import njit
from utils import *

from IPython.display import clear_output

from sklearn.preprocessing import MinMaxScaler

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# TF
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [12]:
N_FOLD = 10
N_MINS = 5
MIN_SIZE = 600 // N_MINS

SOL_NAME = '601-1dCNN'
DATA_NAME = '601'
mkdir(f'./models/{SOL_NAME}/')

In [13]:
# CONSTANT
MEAN = -5.762330803300896
STD = 0.6339307835941186
EPS = 1e-9

In [14]:
# get ids
list_stock_id = get_stock_id()
list_time_id = get_time_id()

# Functions

In [15]:
def transform_target(target):
    return (np.log(target + EPS) - MEAN) / STD

def inverse_target(target):
    return np.exp(MEAN + STD * target) - EPS

def np_rmspe(y_true, y_pred):
    y_true = inverse_target(y_true)
    y_pred = inverse_target(y_pred)
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

def mspe_loss(y_true, y_pred):
    y_true = K.exp(MEAN + STD * y_true) - EPS
    y_pred = K.exp(MEAN + STD * y_pred) - EPS
    return K.sqrt(K.mean(K.square((y_true - y_pred) / y_true)))

In [16]:
def create_1dcnn(num_columns, num_labels, learning_rate):
    # input
    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    # stock embedding
    stock_inp = tf.keras.layers.Input(shape=(1,))
    stock_embedded = tf.keras.layers.Embedding(127, 24, input_length=1, name='stock_embedding')(stock_inp)
    stock_flattened = tf.keras.layers.Flatten()(stock_embedded)
    
    x = tf.keras.layers.Concatenate()([stock_flattened, x])
    # 1dcnn
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Reshape((16, 16))(x)
    x = tf.keras.layers.Conv1D(filters=12,
                      kernel_size=2,
                      strides=1,
                      activation='swish')(x)
    x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
    x = tf.keras.layers.Flatten()(x)
    
    # ffn
    for i in range(3):
        x = tf.keras.layers.Dense(64 // (2 ** i), activation='swish')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.GaussianNoise(0.01)(x)
        x = tf.keras.layers.Dropout(0.20)(x)
    x = tf.keras.layers.Dense(num_labels)(x)
    
    model = tf.keras.models.Model(inputs=[stock_inp, inp], outputs=x)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=mspe_loss,
    )
    return model

# Loading data

In [17]:
# train
df_train = dt.fread(f'./dataset/train_{DATA_NAME}_NN.csv').to_pandas()
fea_cols = [f for f in df_train if f not in ['time_id', 'target', 'pred_NN', 'stock_id', 'row_id']]

# result
df_result = dt.fread('./dataset/train.csv').to_pandas()
df_result = gen_row_id(df_result)

In [18]:
df_train['target'] = transform_target(df_train['target'])
df_train = gen_row_id(df_train)

# Evaluation

In [19]:
batch_size = 1024
hidden_units = [64, 32, 16]
dropout_rates = 0
learning_rate = 6e-3
epochs = 1000

list_seeds = [0, 11, 42, 777, 2045]

In [20]:
list_rmspe = []
for i_seed, seed in enumerate(list_seeds):
    df_train = add_time_fold(df_train, N_FOLD, seed=seed)
    list_rmspe += [[]]
    for i_fold in range(N_FOLD):
        gc.collect()
        df_tr = df_train.loc[df_train.fold!=i_fold]
        df_te = df_train.loc[df_train.fold==i_fold]

        S_train = df_tr['stock_id'].values
        X_train = df_tr[fea_cols].values
        y_train = df_tr[['target']].values
        S_test = df_te['stock_id'].values
        X_test = df_te[fea_cols].values
        y_test = df_te[['target']].values
        idx_test = df_train.loc[df_train.fold==i_fold].index
        print(f'Fold {i_seed+1}/{len(list_seeds)} | {i_fold+1}/{N_FOLD}', X_train.shape, X_test.shape)

        scaler = MinMaxScaler(feature_range=(-1, 1))         
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        save_pickle(scaler, f'./models/{SOL_NAME}/minmax_scaler_{i_seed}_{i_fold}.pkl')

        # Callbacks
        ckp_path = f'./models/{SOL_NAME}/model_{i_seed}_{i_fold}.hdf5'
        rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_delta=1e-5, verbose=2)
        es = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=31, restore_best_weights=True, verbose=2)

        model = create_1dcnn(X_train.shape[1], 1, learning_rate) 
        history = model.fit([S_train, X_train], y_train, 
            epochs=epochs,
            validation_data=([S_test, X_test], y_test),
            validation_batch_size=len(y_test),
            batch_size=batch_size,
            verbose=2,
            callbacks=[rlr, es]
        )
        # model = tf.keras.models.load_model(ckp_path, custom_objects={'mspe_loss': mspe_loss})

        y_pred = model.predict([S_test, X_test], batch_size=len(y_test))
        curr_rmspe = np_rmspe(y_test, y_pred)
        list_rmspe[-1] += [curr_rmspe]
        model.save(ckp_path)
        # generate and save preds
        df_result.loc[idx_test, f'pred_{i_seed}'] = inverse_target(y_pred)
        clear_output()
        print(list_rmspe)

[[0.21125893740946672, 0.21921254035130425, 0.21426420009210184, 0.21122205321353793, 0.21218176512217135, 0.20920462760806532, 0.20984152171058973, 0.21499477093960798, 0.21208924262773357, 0.2221464443546021], [0.21719817110694553, 0.20816893703003794, 0.21018561604988945, 0.21911470939871205, 0.22045467534847865, 0.21012883867875679, 0.2114927599304189, 0.21262690559291125, 0.20766665571591256, 0.2077560869677758], [0.21299586629277775, 0.21651061506758473, 0.21207427815064542, 0.21216163963447918, 0.2050373804543429, 0.22222215603823922, 0.21421396862176065, 0.2084265048780881, 0.20848046047888283, 0.21764033339139302], [0.20967480092926905, 0.207623668183623, 0.20926477819180855, 0.2189151161501688, 0.21331391181862583, 0.21170569396667063, 0.21488084193021806, 0.2066527675548817, 0.21811033635148433, 0.223154126072629], [0.21001562969504772, 0.20845211498015428, 0.21214946940067222, 0.20998114441265056, 0.21755071407616572, 0.21267436216055244, 0.2112806484604464, 0.2149679184186

In [23]:
df_result.to_csv(f'./results/{SOL_NAME}.csv', index=False)

In [24]:
for i in range(len(list_seeds)):
    print(i, rmspe(df_result['target'], df_result[f'pred_{i}']))
print('All: ', rmspe(df_result['target'], df_result[[f'pred_{i}' for i in range(len(list_seeds))]].mean(axis=1)))

0 0.21367807400716804
1 0.2125278967970332
2 0.213029445369315
3 0.2133912249112298
4 0.21252088476833877
All:  0.2090298992545508
